In [1]:
# Read all tables in a root directory, get the columns in each

In [2]:
#TODO: Fix the readAttributes so it always returns list (even if there is only one attribute)

In [1]:
import wctablereader as tablereader
from glob import glob
import pandas as pd

import importlib

In [2]:
importlib.reload(tablereader)
tb = tablereader.TableReader("../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/BulkMolecules")
print(tb.columnNames())
df = tb.as_pandas("counts")

['atpRequested', 'atpAllocatedFinal', 'atpAllocatedInitial', 'counts']


In [3]:
#Sweep some output, see if/where there are problems
import warnings

tables = glob("../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/*")
for table in tables:
    first_error = True
    try: 
        tb = tablereader.TableReader(table)
        print(f"Success opening {table}")
    except: 
        print(f"!!!!Error opening {table}")
        continue
    for column in tb.columnNames():
        try: 
            warnings.filterwarnings("error")
            df = tb.as_pandas(column)
            warnings.filterwarnings("ignore")
        except Exception as e: 
            if first_error:
                print(f"Error reading column in {table}")
                first_error = False
            print(f"\tError on '{column}'")
            print("\t", e)

Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/Mass
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/FBAResults
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/ReplicationData
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/BulkMolecules
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/RnaDegradationListener
Error reading column in ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/RnaDegradationListener
	Error on 'countRnaDegraded'
	 No attributes match shape for countRnaDegraded.  Returning data without labels.
Success opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/RibosomeData
!!!!Error opening ../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/Daughter2
Success open

In [5]:
tb = tablereader.TableReader("../../WCOutputs/manual/wildtype_000000/000000/generation_000000/000000/simOut/FBAResults")
tb.columnNames()

['simulationStep',
 'targetConcentrations',
 'succinate_flux_sensitivity',
 'externalExchangeFluxes',
 'time',
 'objectiveValue',
 'reducedCosts',
 'reactionFluxes',
 'isocitrate_flux_sensitivity',
 'homeostaticObjectiveValues',
 'kineticObjectiveValues',
 'deltaMetabolites',
 'shadowPrices']

In [8]:
df = {}
for col in tb.columnNames():
    df[col] = tb.as_pandas( col )
    df[col].index.name = col
    display(df[col])

,0
simulationStep,
0,0
1,1
2,2
3,3
4,4
...,...
2877,2877
2878,2878
2879,2879


,2-3-DIHYDROXYBENZOATE[c],2-KETOGLUTARATE[c],4-hydroxybenzoate[c],ACETOACETYL-COA[c],ACETYL-COA[c],ACETYL-P[c],ADENINE[c],ADENOSINE[c],ADP-D-GLUCOSE[c],ADP[c],...,UDP[c],UMP[c],UNDECAPRENYL-DIPHOSPHATE[c],URIDINE[c],UTP[c],VAL[c],WATER[c],ZN+2[c],ZN+2[p],glycogen-monomer[c]
targetConcentrations,,,,,,,,,,,,,,,,,,,,,
0,0.00,0.00,0.000,0.000,0.00,0.0,0.0000,0.00000,0.0000,0.00,...,0.0,0.000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000
1,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,0.00013,0.0043,0.56,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
2,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,0.00013,0.0043,0.56,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
3,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,0.00013,0.0043,0.56,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
4,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,0.00013,0.0043,0.56,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,0.00013,0.0043,0.56,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
2878,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,0.00013,0.0043,0.56,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786
2879,0.14,0.44,0.052,0.022,0.61,1.1,0.0015,0.00013,0.0043,0.56,...,1.8,0.024,0.025115,2.1,8.3,4.0,42742.159312,0.168848,0.1,66.026786


""
succinate_flux_sensitivity
0
1
2
3
4
...
2877
2878
2879


,ACET[p],AMMONIUM[c],BETAINE[p],BUTANAL[c],CA+2[p],CARBON-DIOXIDE[p],CARBON-MONOXIDE[p],CH33ADO[c],CL-[p],CO+2[p],...,S-ADENOSYL-4-METHYLTHIO-2-OXOBUTANOATE[c],SUC[p],SULFATE[p],THYMINE[p],UNDECAPRENYL-DIPHOSPHATE[p],URACIL[p],UREA[p],WATER[p],XANTHINE[p],ZN+2[p]
externalExchangeFluxes,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
1,-0.0,-12.906195,0.0,0.0,-0.219226,22.165272,0.007560,-0.0,-0.209684,-0.009183,...,0.001700,-0.0,-2.051496,0.0,0.000000,-0.0,0.0,287.528753,-0.0,-0.021913
2,-0.0,-14.038226,-0.0,-0.0,0.000042,14.982359,0.000068,-0.0,0.000015,-0.000059,...,0.000000,-0.0,-0.119526,-0.0,0.000267,-0.0,-0.0,197.452434,-0.0,-0.000030
3,-0.0,-6.054397,-0.0,-0.0,-0.000032,29.206674,-0.000000,-0.0,0.000015,0.000015,...,0.000000,-0.0,-0.116480,-0.0,0.000468,-0.0,-0.0,161.462264,-0.0,0.000044
4,-0.0,-6.884775,-0.0,-0.0,0.000041,37.138720,-0.000000,-0.0,0.000014,-0.000059,...,0.000000,-0.0,-0.110465,-0.0,0.000468,-0.0,-0.0,99.973080,-0.0,0.000044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,-0.0,-7.829511,-0.0,-0.0,-0.007475,29.625417,0.000262,-0.0,-0.007198,-0.000315,...,0.000062,-0.0,-0.174776,-0.0,0.010496,-0.0,-0.0,-69.951339,-0.0,-0.000747
2878,-0.0,-7.962685,-0.0,-0.0,-0.007470,29.671164,0.000254,-0.0,-0.007189,-0.000321,...,0.000050,-0.0,-0.178593,-0.0,0.010485,-0.0,-0.0,-69.706458,-0.0,-0.000743
2879,-0.0,-7.927950,-0.0,-0.0,-0.007491,29.691394,0.000257,-0.0,-0.007213,-0.000318,...,0.000057,-0.0,-0.181078,-0.0,0.010527,-0.0,-0.0,-70.104030,-0.0,-0.000750


,0
time,
0,0.000000
1,0.200000
2,0.400000
3,0.600000
4,0.800000
...,...
2877,2565.606250
2878,2566.499219
2879,2567.392187


...

""
isocitrate_flux_sensitivity
0
1
2
3
4
...
2877
2878
2879


,2-3-DIHYDROXYBENZOATE[c],2-KETOGLUTARATE[c],4-hydroxybenzoate[c],ACETOACETYL-COA[c],ACETYL-COA[c],ACETYL-P[c],ADENINE[c],ADENOSINE[c],ADP-D-GLUCOSE[c],ADP[c],...,UDP[c],UMP[c],UNDECAPRENYL-DIPHOSPHATE[c],URIDINE[c],UTP[c],VAL[c],WATER[c],ZN+2[c],ZN+2[p],glycogen-monomer[c]
homeostaticObjectiveValues,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000e+00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.001591,0.0,0.0,8.340194e-04
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.001655,0.0,0.0,0.000000e+00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.001703,0.0,0.0,-5.618557e-09
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.001723,0.0,0.0,-9.248314e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000e+00
2878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000e+00
2879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000e+00


,1.1.1.215-RXN,1.1.1.262-RXN,1.1.1.39-RXN,1.1.1.83-RXN,1.8.4.13-RXN-MET/Ox-Thioredoxin/WATER//CPD-8989/Red-Thioredoxin.51.__EG11433-MONOMER,1.8.4.14-RXN-MET/Ox-Thioredoxin/WATER//CPD-8990/Red-Thioredoxin.51.__EG12394-MONOMER,1.8.4.14-RXN-MET/Ox-Thioredoxin/WATER//CPD-8990/Red-Thioredoxin.51.__G7005-MONOMER,2-DEHYDROPANTOATE-REDUCT-RXN__2-DEHYDROPANTOATE-REDUCT-MONOMER,2-DEHYDROPANTOATE-REDUCT-RXN__CPLX0-7643,2-OXOPENT-4-ENOATE-HYDRATASE-RXN,...,UDPNACETYLGLUCOSAMACYLTRANS-RXN-R-3-hydroxymyristoyl-ACPs/UDP-N-ACETYL-D-GLUCOSAMINE//UDP-OHMYR-ACETYLGLUCOSAMINE/ACP.86.,UDPNACETYLGLUCOSAMENOLPYRTRANS-RXN,UDPNACETYLMURAMATEDEHYDROG-RXN,UGD-RXN,UNDECAPRENYL-DIPHOSPHATASE-RXN__PGPPHOSPHAB-MONOMER,URA-PHOSPH-RXN,UROGENIIISYN-RXN,URPHOS-RXN (reverse),XANPRIBOSYLTRAN-RXN,XYLULOKIN-RXN
kineticObjectiveValues,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,0.0,17.088456,1.0,1.0,1.0,1.0,1.0,0.936256,0.000000,1.0,...,1.000000,1.000000,1.000000,19.037087,1.000000,0.0,0.999990,0.0,1.000000,1.0
2,0.0,0.947322,1.0,1.0,1.0,1.0,1.0,1.000000,0.995004,1.0,...,0.873780,0.981278,0.992016,0.993350,0.631569,0.0,0.999990,0.0,0.989216,1.0
3,0.0,1.000000,0.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,...,46.950809,0.967201,0.986013,1.000000,0.354535,0.0,0.999990,0.0,0.985932,1.0
4,0.0,1.000000,0.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0,...,1.000000,0.967201,0.986013,1.000000,0.354535,0.0,0.999990,0.0,0.970741,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,0.0,0.436408,0.0,1.0,1.0,1.0,1.0,1.000000,0.541219,1.0,...,1.099964,0.189672,0.642045,0.333973,0.000000,0.0,0.997746,0.0,0.000000,1.0
2878,0.0,0.435892,0.0,1.0,1.0,1.0,1.0,1.000000,0.541025,1.0,...,1.098463,0.190910,0.642341,0.337959,0.000000,0.0,0.997803,0.0,0.000000,1.0
2879,0.0,0.433482,0.0,1.0,1.0,1.0,1.0,1.000000,0.540450,1.0,...,1.105757,0.187516,0.640841,0.339279,0.000000,0.0,0.997752,0.0,0.000000,1.0


,2-3-DIHYDROXYBENZOATE[c],2-KETOGLUTARATE[c],4-hydroxybenzoate[c],ACETOACETYL-COA[c],ACETYL-COA[c],ACETYL-P[c],ADENINE[c],ADENOSINE[c],ADP-D-GLUCOSE[c],ADP[c],...,UDP[c],UMP[c],UNDECAPRENYL-DIPHOSPHATE[c],URIDINE[c],UTP[c],VAL[c],WATER[c],ZN+2[c],ZN+2[p],glycogen-monomer[c]
deltaMetabolites,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,154.0,482.0,58.0,453.0,668.0,1205.0,2.0,0.0,5.0,614.0,...,1972.0,27.0,28.0,2301.0,10810.0,8221.0,-3037975.0,185.0,110.0,31951.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,...,-1.0,-636.0,0.0,-1.0,1701.0,3786.0,-2048778.0,1.0,0.0,40352.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-91.0,...,0.0,-542.0,0.0,0.0,1722.0,3770.0,-1541165.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,-308.0,0.0,0.0,1703.0,3562.0,-659573.0,0.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,46.0,143.0,16.0,7.0,199.0,358.0,1.0,1.0,1.0,181.0,...,586.0,-4501.0,9.0,683.0,18088.0,31693.0,13544696.0,55.0,33.0,21495.0
2878,45.0,144.0,17.0,8.0,198.0,358.0,0.0,-1.0,2.0,184.0,...,587.0,-4206.0,8.0,684.0,17697.0,32302.0,13545455.0,55.0,32.0,21494.0
2879,46.0,143.0,17.0,7.0,199.0,359.0,1.0,0.0,1.0,182.0,...,588.0,-5520.0,8.0,685.0,18252.0,33141.0,13577688.0,55.0,33.0,21567.0


,2-3-DIHYDROXYBENZOATE[c],2-KETOGLUTARATE[c],4-hydroxybenzoate[c],ACETOACETYL-COA[c],ACETYL-COA[c],ACETYL-P[c],ADENINE[c],ADENOSINE[c],ADP-D-GLUCOSE[c],ADP[c],...,UDP[c],UMP[c],UNDECAPRENYL-DIPHOSPHATE[c],URIDINE[c],UTP[c],VAL[c],WATER[c],ZN+2[c],ZN+2[p],glycogen-monomer[c]
shadowPrices,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
1,0.016521,0.009348,0.066231,0.167616,0.163322,4.408507e-03,0.090266,0.102479,0.118586,0.103440,...,0.015637,0.014911,0.341429,0.014142,0.016364,0.013276,2.339608e-05,8.557589e-05,8.557589e-05,0.015145
2,0.012988,0.008815,-0.034717,0.132158,0.128033,4.177744e-03,0.093493,0.103089,0.115559,0.103780,...,0.012339,0.011953,0.102618,0.011497,0.012726,0.010965,2.339608e-05,8.557589e-05,8.557589e-05,0.011779
3,0.002998,0.001984,-0.045068,-0.712594,-0.713439,8.946009e-04,0.101186,0.103628,0.088816,0.103961,...,0.004325,0.004120,0.119981,0.003890,0.004531,0.002913,2.339608e-05,6.006405e-05,8.557589e-05,-0.015145
4,0.002976,0.001946,-0.045094,-0.712592,-0.713438,1.064872e-03,0.101207,0.103632,0.088818,0.103963,...,0.004789,0.004585,0.119642,0.004358,0.004993,0.002899,2.339608e-05,6.055000e-05,8.557589e-05,-0.015145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877,-0.000077,0.000005,-0.022488,0.030330,0.030330,-3.818907e-08,0.027716,0.027716,0.027850,0.027870,...,0.000049,0.000049,0.057316,0.000048,0.000049,0.000078,2.656466e-08,9.642317e-08,9.642317e-08,-0.000020
2878,-0.000077,0.000005,-0.022493,0.030344,0.030345,-3.818907e-08,0.027722,0.027722,0.027856,0.027876,...,0.000049,0.000048,0.057329,0.000048,0.000049,0.000078,2.656466e-08,9.642317e-08,9.642317e-08,-0.000020
2879,-0.000077,0.000005,-0.022498,0.030357,0.030357,-3.818907e-08,0.027728,0.027729,0.027862,0.027883,...,0.000049,0.000048,0.057340,0.000048,0.000049,0.000078,2.656466e-08,9.642317e-08,9.642317e-08,-0.000020
